In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

## Resilient distributed systems

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/04 10:09:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [4]:
from datetime import datetime

In [5]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

## filter out old data

In [6]:
rows = rdd.take(10)

In [7]:
row = rows[0]

In [8]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)

In [9]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [10]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [11]:
from collections import namedtuple

In [12]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [13]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )


In [14]:
rdd\
    .filter(filter_outliers)\
    .map(prepare_for_grouping)\
    .reduceByKey(calculate_revenue)\
    .map(unwrap)\
    .toDF()\
    .show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-20 15:00:00|  67|              79.5|    3|
|2020-01-05 16:00:00|  82| 500.4700000000002|   33|
|2020-01-16 08:00:00|  41| 736.1399999999996|   54|
|2020-01-20 15:00:00|  75| 608.9999999999999|   47|
|2020-01-17 21:00:00|  74|            594.87|   39|
|2020-01-03 09:00:00|  61|            142.21|    9|
|2020-01-13 19:00:00| 223|213.83000000000004|   19|
|2020-01-16 13:00:00|  14|163.73999999999998|    6|
|2020-01-12 15:00:00|  82| 265.7900000000001|   29|
|2020-01-08 16:00:00|  72|            130.67|    7|
|2020-01-03 11:00:00|  51| 305.9599999999999|   10|
|2020-01-30 09:00:00|  22|114.80000000000001|    4|
|2020-01-31 18:00:00| 260| 439.3100000000002|   27|
|2020-01-10 20:00:00|  66|            405.88|   21|
|2020-01-31 21:00:00|  41| 588.1600000000001|   40|
|2020-01-23 22:00:00| 166|            297.84|   18|
|2020-01-30 

In [15]:
from pyspark.sql import types

In [16]:

result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [17]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [18]:
df_result.show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-20 15:00:00|  67|              79.5|    3|
|2020-01-05 16:00:00|  82| 500.4700000000002|   33|
|2020-01-16 08:00:00|  41| 736.1399999999996|   54|
|2020-01-20 15:00:00|  75| 608.9999999999999|   47|
|2020-01-17 21:00:00|  74|            594.87|   39|
|2020-01-03 09:00:00|  61|            142.21|    9|
|2020-01-13 19:00:00| 223|213.83000000000004|   19|
|2020-01-16 13:00:00|  14|163.73999999999998|    6|
|2020-01-12 15:00:00|  82| 265.7900000000001|   29|
|2020-01-08 16:00:00|  72|            130.67|    7|
|2020-01-03 11:00:00|  51| 305.9599999999999|   10|
|2020-01-30 09:00:00|  22|114.80000000000001|    4|
|2020-01-31 18:00:00| 260| 439.3100000000002|   27|
|2020-01-10 20:00:00|  66|            405.88|   21|
|2020-01-31 21:00:00|  41| 588.1600000000001|   40|
|2020-01-23 22:00:00| 166|            297.84|   18|
|2020-01-30 

In [19]:
df_result.write.parquet('tmp/green-revenue', mode = 'overwrite')

In [20]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [25]:
duration_rdd.take(5)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, DOLocationID=130, trip_distance=12.77),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, DOLocationID=39, trip_distance=8.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, DOLocationID=157, trip_distance=1.27),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, DOLocationID=83, trip_distance=1.25),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, DOLocationID=42, trip_distance=1.84)]

In [21]:
import pandas as pd

In [22]:
rows = duration_rdd.take(10)
df = pd.DataFrame(rows, columns=columns)

In [23]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [26]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [27]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row


In [28]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [31]:
def infinite_seq():
    i=0

    while True:
        yield i
        i = i + 1

        if i>15:
            break

In [32]:
seq = infinite_seq()    

In [33]:
list(seq)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [ ]:
for i in seq:
    print(i)

    if i >= 10:
        break 

In [34]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|63.849999999999994|
|              40.0|
|              6.35|
|              6.25|
| 9.200000000000001|
|               3.8|
|16.599999999999998|
|             11.05|
|               4.5|
|              30.5|
|               8.7|
|5.8999999999999995|
|              11.0|
|              15.2|
|              4.25|
|25.299999999999997|
|7.8500000000000005|
|              34.0|
| 5.300000000000001|
|              6.15|
+------------------+
only showing top 20 rows

